In [16]:
import pandas as pd
import numpy as np
import glob
import datetime

In [17]:
icews_cartels = pd.read_csv('../data/icews_cartels.csv', parse_dates=['event_date'],dtype={
       'event_id':'int','intensity':'float','story_id':'int','sentence_number':'int'})

In [18]:
icews_cartels["event_text"].value_counts()

Use unconventional violence                            993
Use conventional military force                        354
fight with small arms and light weapons                101
Abduct, hijack, or take hostage                         67
Assassinate                                             32
Occupy territory                                        16
Kill by physical assault                                14
Torture                                                  8
Engage in mass killings                                  7
Conduct suicide, car, or other non-military bombing      2
Employ aerial weapons                                    1
fight with artillery and tanks                           1
Name: event_text, dtype: int64

In [19]:
#viewing different events to see major sources of event activities
mex_occupy = icews_cartels.loc[icews_cartels["event_text"].isin(['Occupy territory'])]
mex_occupy.head(10)

,Unnamed: 0,Unnamed: 0.1,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,...,story_id,sentence_number,publisher,city,district,provice,country,lat,lon,event_year
225,13422718,13422718,20774982,2014-01-16,Knights Templar Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,32588437,5,Associated Press Newswires,Tancitaro,NaN,Estado de Michoacan de Ocampo,Mexico,19.3377,-102.3620,2014
265,16337700,16337700,35016053,2019-10-22,Sinaloa Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,50539757,4,Reuters News,Culiacan,NaN,Estado de Sinaloa,Mexico,24.8074,-107.3970,2019
267,16337752,16337752,35017144,2019-10-22,Sinaloa Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,50539869,2,Reuters News,Culiacan,NaN,Estado de Sinaloa,Mexico,24.8074,-107.3970,2019
435,12402962,12402962,18977505,2012-08-23,Miguel Treviño Morales,"Criminals / Gangs,Dissident",Mexico,Occupy territory,192,-9.5,...,17200980,3,Associated Press Newswires,Mexico City,NaN,Distrito Federal,Mexico,19.4285,-99.1277,2012
491,4302126,4302126,6280294,2003-07-15,Drug Gang (Mexico),"Criminals / Gangs,Dissident",Mexico,Occupy territory,192,-9.5,...,13668451,2,Agence France-Presse,Nuevo Laredo,NaN,Estado de Tamaulipas,Mexico,27.4763,-99.5164,2003
524,7355038,7355038,10579504,2006-08-20,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,14650943,2,El Universal,Sinaloa,NaN,Estado de Chiapas,Mexico,15.8931,-92.1298,2006
525,7360972,7360972,10585402,2006-08-22,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,14653773,2,El Universal,Sinaloa,NaN,Estado de Chiapas,Mexico,15.8931,-92.1298,2006
669,9835927,9835927,14221307,2009-01-29,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15098980,3,Mural,Guadalajara,NaN,Estado de Jalisco,Mexico,20.6668,-103.3920,2009
670,9838931,9838931,14226178,2009-01-30,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15099439,4,Reforma,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009
706,10250534,10250534,14759579,2009-06-29,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15118826,2,El Norte,Guanajuato,NaN,Estado de Chiapas,Mexico,15.1059,-92.2526,2009


In [20]:
#explore source sectors for consolidation
icews_cartels["source_sectors"].unique()

array(['Dissident,Criminals / Gangs', 'Criminals / Gangs,Dissident', nan,
       'General Population / Civilian / Social,Social',
       'Social,General Population / Civilian / Social',
       'Protestors / Popular Opposition / Mobs,Dissident',
       'Government,Military', 'Police,Government', 'Government,Police',
       'Government,Army,Military', 'Government,Marines,Military',
       'Military,Government', 'Unidentified Forces', 'Social,Business',
       'Military Intelligence,Military,Government',
       'Navy,Military,Government', 'Military,Marines,Government',
       'Marines,Government,Military', 'Military,Government,Marines',
       'Marines,Military,Government',
       'Justice / Law Ministry,Executive,Government,Drugs Ministry',
       'Local,Provincial,Government', 'Government',
       'Government,Police,National / Border Divisions',
       'Government,Executive,Foreign Ministry',
       'Government,Military,Army',
       'Military,Government,Military Intelligence',
       '

COMBINE AND CLEAN SOURCE SECTORS

In [21]:
source_sectors = {
    "Government,Police": "Government/Military/Police",
    "Police,Government": "Government/Military/Police",
    "Government,Police,National / Border Divisions": "Government/Military/Police",
    "Government,Military": "Government/Military/Police",
    "Military,Government": "Government/Military/Police",
    "Government,Marines,Military": "Government/Military/Police",
    "Military Intelligence,Military,Government": "Government/Military/Police",
    "Navy,Military,Government": "Government/Military/Police",
    "Army,Government,Military": "Government/Military/Police",
    "Military,Government,Army": "Government/Military/Police",
    "Government,Army,Military": "Government/Military/Police",
    "Government,Military,Army": "Government/Military/Police",
    "Marines,Government,Military": "Government/Military/Police",
    "Military,Government,Military Intelligence": "Government/Military/Police",
    "Army,Military,Government": "Government/Military/Police",
    "Military,Navy,Government": "Government/Military/Police",
    "Government,Military,Navy": "Government/Military/Police",
    "Marines,Military,Government": "Government/Military/Police",
    "Military,Government,Navy": "Government/Military/Police",
    "Government,Navy,Military": "Government/Military/Police",
    "Military,Marines,Government": "Government/Military/Police",
    "Government,Military,Marines": "Government/Military/Police",
    "Military,Government,Marines": "Government/Military/Police",
    "Justice / Law Ministry,Executive,Government,Drugs Ministry": "Government/Military/Police",
    "Local,Provincial,Government": "Government/Military/Police",
    "Government,Executive,Foreign Ministry": "Government/Military/Police",
    "Social,General Population / Civilian / Social": "Civilian",
    "General Population / Civilian / Social,Social": "Civilian",
    "Dissident,Criminals / Gangs": "Criminals/Dissidents",
    "Social,Business": "Civilian",
    "Protestors / Popular Opposition / Mobs,Dissident": "Civilian",
    "Criminals / Gangs,Dissident": "Criminals/Dissidents"}
icews_cartels = icews_cartels.replace(source_sectors)

In [22]:
#explore source sectors for consolidation
icews_cartels["source_sectors"].unique()

array(['Criminals/Dissidents', nan, 'Civilian',
       'Government/Military/Police', 'Unidentified Forces', 'Government'],
      dtype=object)

In [24]:
#explore source sectors for consolidation
icews_cartels["source_name"].unique()

array(['Juárez Cartel', 'Tijuana Cartel', 'Sinaloa Cartel',
       'Los Zetas Cartel', 'Gulf Cartel', 'Beltrán-Leyva Cartel',
       'La Familia Michoacana Cartel', 'Knights Templar Cartel',
       'La Línea Cartel', 'Attacker (Mexico)', 'Citizen (Mexico)',
       'Criminal (Mexico)', 'Citizen (United States)',
       'Drug Gang (Mexico)', 'Joaquín Guzmán Loera', 'Mexico',
       'Mob (Mexico)', 'Military (Mexico)', 'Police (Mexico)',
       'Barrio Azteca Drug Gang', 'Military Personnel (United States)',
       'Military Personnel - Special (Mexico)', 'Mexican Army',
       'Edgar Valdez Villarreal', 'Marine Corp (Mexico)',
       'Armed Gang (Mexico)', 'Business (Mexico)',
       'Miguel Treviño Morales', 'Secret Agent (Mexico)', 'Mexican Navy',
       'Militia (Mexico)', 'Military Personnel (Mexico)', 'Men (Mexico)',
       'Drug Dealer (Mexico)', 'Drug Gang (Brazil)',
       'Drug Enforcement Administration', 'Drug Dealer (Brazil)',
       'Governor (Mexico)', 'Police (Brazil)', 'U

In [25]:
icews_cartels["target_name"].unique()

array(['Citizen (Mexico)', 'Men (Mexico)', 'Gulf Cartel', 'Mexico',
       'Drug Gang (Mexico)', 'Joaquín Guzmán Loera', 'Mob (Mexico)',
       'Criminal (Mexico)', 'Police (Mexico)', 'Military (Mexico)',
       'Sinaloa Cartel', 'Los Zetas Cartel', 'Mexican Navy',
       'Other Authorities / Officials (Mexico)', 'Juárez Cartel',
       'Children (Mexico)', 'Federal Police',
       'La Familia Michoacana Cartel', 'Barrio Azteca Drug Gang',
       'Farm Worker (Mexico)', 'Women (Mexico)',
       'Illegal Immigrant (Mexico)', 'Immigrants (Mexico)',
       'Marine Corp (Mexico)', 'Jaime Zapata', 'Detainee (Mexico)',
       'Criminal (Gulf Cartel)', 'Violent Group (Mexico)',
       'Business (Mexico)', 'Immigrants (Brazil)',
       'Secret Agent (Mexico)', 'Governor (Mexico)', 'Central America',
       'Military Personnel (Mexico)',
       'Military Personnel - Special (Mexico)', 'La Línea Cartel',
       'Immigrants (Latin America)', 'Party Member (Colombia)',
       'Gente Nueva', 'Tijua

In [9]:
#mex_180.to_csv('../data/icews_mexico_180.csv')

In [10]:
# what to do with events without a longitude latitude